## Creating Biased Datasets Using Simstudy

In this notebook, we will be using Simstudy to simulate a dataset in which certain outcomes are biased, conditional on certain characteristics. We aim to demonstrate how Simstudy can be used to develop accelerators, as well as Trustworthy AI demos. By specifying the relationships between inputs and outputs ahead of time, we can see how well different algorithms do at estimating these relationships.

In [1]:
cd ../

/Users/davidcruz/Desktop/pysimstudy


In [2]:
import math
import pandas as pd
import numpy as np
import random
from scipy import special
import seaborn as sns
import matplotlib.pyplot as plt

from py_scripts.generate_dist import *
from py_scripts.group_data import *
from py_scripts.define_data import *
from py_scripts.generate_data import *
from py_scripts.add_data import *
from py_scripts.asserts import *

import statsmodels.api as sm

np.random.seed(42)

def coefasProb(coef):
    return np.exp(coef) / (1 + np.exp(coef))

Let's start with the building blocks of the pysimstudy package. 

The goal of this first example is to display the core steps in generating data with pysimstudy, while also exploring the art of whats possible through interesting questions of statistical nature. We will work to create a dataset of people represented by an arbitrary id, along with their income. The target variable will be approval, which will be biased to have more approvals if your income is higher. Our goal is to show that we can hard-code bias into an approval rating process based on a random variable.

The first step is to define a _definitions_ data table in which we describe the variables we want to generate. We pass arguments to define the generated variable's name (varname), as well as its statistical parameters. The _formula_ argument refers to the variable's mean, or expected value, and _variance_ is self evident. The _formula_ argument allows you to pass an equation to generate your data. In the example below, we want our __income__ variable to be centered at 5000, but as we will see later on, we can also make the formula dependent on previously written data definitions to generate complex dependencies in the synthetic data. Note that the _variance_ argument is squared in this case, since the standard deviation is the square-root of variance.

In [3]:
ddf = defData(varname = "income", formula=5000,
             variance="1000**2", dist="normal")
# We wil generate our first dataset with the following line of code
# this wil allow us to implement conditions onto the generated data
gdf = genData(10000, ddf)
# Now we can create conditions to influence our data
defC = defCondition(condition = "income >= 6000", formula = "0.9",
                    dist = "binary")

defC = defCondition(defC, condition = "(income < 6000) & (income >= 4000)", formula = 0.65,
                    dist = "binary")

defC = defCondition(defC, condition = "(income < 4000)", formula = 0.4,
                    dist = "binary")

ddf

,varname,formula,variance,dist,link
0,income,5000,1000**2,normal,identity


Now that we created the conditions for our dataset, we can generate observations based on the previously defined conditions.

In [4]:
gdfb = addCondition(defC, gdf, newvar="approval_bias")

In [5]:
gdfb.head()

,id,income,approval_bias
0,3,6523.029856,1
1,6,6579.212816,1
2,20,6465.648769,1
3,31,6852.278185,1
4,47,6057.122226,1


Notice how each of the conditions we defined combine into one condition table

In [ ]:
defC

pysimstudy then took our conditions and resulted in a newly generated column with a binary distribution of observations with positive observations equal to the decimal passed into formula argument.

In [7]:
gdfb[gdfb['income'] >= 6000]['approval_bias'].value_counts(normalize=True)

1    0.89705
0    0.10295
Name: approval_bias, dtype: float64

This is the same for the other brackets

In [8]:
gdfb[(gdfb['income'] < 6000) & (gdfb['income'] >=4000)]['approval_bias'].value_counts(normalize=True)

1    0.641349
0    0.358651
Name: approval_bias, dtype: float64

In [9]:
gdfb[gdfb['income'] < 4000]['approval_bias'].value_counts(normalize=True)

0    0.565217
1    0.434783
Name: approval_bias, dtype: float64

In [10]:
gdfb['approval_bias'].value_counts(normalize=True)

1    0.6493
0    0.3507
Name: approval_bias, dtype: float64

Sometimes it's not that clear to observe how biased data presents itself without using a nice bar graph

In [ ]:
gdfb['approval_bias'].hist();


We then define our target variable, __approval__. We add a new row to the data definitions table created in the previous lines of code by passing _df_ as the first argument. The _formula_ is defined as follows: the probability of an individual being approved is 50% +/- their relative income. In other words, inviduals who are on the higher (lower) end of the income scale will be more (less) likely to receive an approval.


In [ ]:
# df = defData(df, varname = "region", formula="0.333, 0.333, 0.333",
#              variance="red, blue, green", dist="categorical")
             
# ddf = defData(ddf, varname="approval", formula=')', dist='binary')


In [ ]:
gdf = genData(10000, ddf)
# gdf.head()

In [ ]:
sns.displot(gdf['income']);

In [ ]:
gdf.head()


** this can be captioned under a screenshot maybe


In certain cases, the formula and variance arguments are used differently. For instance, when defining a categorical data definition, the _formula_ argument is used to determine the categories' respective probabilities of being drawn, and the _variance_ argument defines the categories' names. 

In [ ]:
# defC = defCondition(condition = "color=='blue'", formula = "0.1", variance = "0.1",
#                     dist = "normal")

# defC = defCondition(defC, condition = "color!='blue'", formula = "0", variance = "0.1",
#                     dist = "normal")

# df3 = addCondition(defC, df2, newvar="income")

## Case 2 - Biased Data

### Sub-Scenario 1 - Direct bias

In this example we introduce categorical distribution named "region". This shows a uniform distribution of observations belonging to one of three regions based on color. Our income is still normally distributed across the population, but there is going to be a protected class that gets "discriminated" against to simulate direct bias. In this case, if you're in the blue region, you will be approved 10% less of the time.

here we demonstrate the defCondition function which is used for conditional distributions ... 

In [ ]:
ddf = defData(ddf, varname = "region", formula="0.333, 0.333, 0.333",
             variance="red, blue, green", dist="categorical")

ddf

In [ ]:
gdf2 = genData(10000, ddf)
gdf2.head()

below shows blue region have 10% less approval, since we have hard coded a 10% less probability of getting a loan approved

In [ ]:
defC = defCondition(condition = "region=='blue'", formula = "0.4+income",
                    dist = "binary")

defC = defCondition(defC, condition = "region!='blue'", formula = "0.5+income",
                    dist = "binary")

gdf2 = addCondition(defC, gdf2, newvar="approval_bias")

In [ ]:
gdf2

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12,6))

sns.histplot(
    data=gdf2, x='approval_bias', hue='region', multiple='dodge',
    # bins=range(1, 110, 10),
    ax=ax[1]
)

sns.histplot(
    data=gdf2, x='approval', hue='region', multiple='dodge',
    # bins=range(1, 110, 10),
    ax=ax[0]
)

### subscenario 2
it is also possible to have indirect bias
here blue region get a 20% less income negative bonus


In [ ]:
defIncomeBiased = defCondition(condition = "region=='blue'", 
                    formula = "0.8*income", variance = "0.1^2",
                    dist = "normal")

In [ ]:
gdf3 = addCondition(defIncomeBiased, gdf2, newvar="income_bias", keepOld=True)

In [ ]:
gdf3.groupby('region')['income'].describe()

In [ ]:
gdf3.groupby('region')['income_bias'].describe()

In [ ]:
gdf.groupby('region')['income_bias'].describe()

In [ ]:
defIndirectBiasApproval = defData(varname = "approval_indirect_bias", dist = "binary", formula = "0.5+(income_bias)",)
gdf3 = addColumns(defIndirectBiasApproval, gdf3)

In [ ]:
gdf3.groupby('region')['approval_indirect_bias'].value_counts(normalize=True)

In [ ]:
gdf3.groupby('region')['approval'].value_counts(normalize=True)

In [ ]:
gdf3['const'] = 1

In [ ]:
X = pd.get_dummies(gdf3.drop(['id', 'region', 'income', 'approval', 'approval_bias', 'approval_indirect_bias'], axis = 1))

mod = sm.GLM(gdf3['approval_indirect_bias'],
             X ,
            family=sm.families.Binomial())

res = mod.fit()

print(res.summary())

In [ ]:
X = pd.get_dummies(gdf3.drop(['id', 'income', 'approval', 'approval_bias', 'approval_indirect_bias'], axis = 1))

mod = sm.GLM(gdf3['approval_indirect_bias'],
             X ,
            family=sm.families.Binomial())

res = mod.fit()

print(res.summary())

In [ ]:
# the coefficient of red as a probability
coefasProb(0.1274)

## Case Study 2: What happens when Blue is a minority category?

In [ ]:
df = defData(varname = "color", formula="0.55, 0.10, 0.35",
             variance="red, blue, green", dist="categorical")
             
df = defData(df, varname = "income", formula=0,
             variance=1, dist="normal")
             
df = defData(df, varname="approval", formula='0.5', dist='binary')

df2 = genData(10000, df)

defC = defCondition(condition = "color=='blue'", formula = "0.4+income/10",
                    dist = "binary")

defC = defCondition(defC, condition = "color!='blue'", formula = "0.5+income/10",
                    dist = "binary")

df3 = addCondition(defC, df2, newvar="approval_bias")

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (10,6))
sns.histplot(
    data=df3, x='income', hue='approval_bias', multiple='dodge',
    bins=range(1, 110, 10),
    ax=ax[0]
)

sns.histplot(
    data=df3, x='income', hue='color', multiple='dodge',
    bins=range(1, 110, 10),
    ax=ax[1]
)

sns.histplot(
    data=df3, x='approval_bias', hue='color', multiple='dodge',
    bins=range(1, 110, 10),
    ax=ax[2]
)

In [ ]:
df3['const'] = 1
mod = sm.GLM(df3['approval_bias'], pd.get_dummies(df3.drop(['id', 'approval_bias'], axis = 1)),
family=sm.families.Binomial())
res = mod.fit()
print(res.summary())

In [ ]:
# the coefficient of blue as a prb
coefasProb(-0.3194)

In [ ]:
# the coefficient of red as a prb
coefasProb(0.0854)

In [ ]:
# the coefficient of green as a prob
coefasProb(0.1364)

## Case Study 3

Let's add some predictor variables

And make a couple of them correlated with color

Then regress approval on these variables, excluding color

In [ ]:
df = defData(varname = "eye_color", formula="0.55, 0.10, 0.35",
             variance="red, blue, green", dist="categorical")

df = defData(df, varname = "gender", formula=0.5,
            dist="binary")
             
df2 = genData(10000, df)

defNeighborhood = defCondition(condition = "eye_color=='blue'", 
                    formula = "0.75, 0.05, 0.05, 0.15",
                    variance = "N1, N2, N3, N4",
                    dist = "categorical")

defNeighborhood = defCondition(defNeighborhood, condition = "eye_color=='red'", 
                    formula = "0.05, 0.55, 0.2, 0.2",
                    variance = "N1, N2, N3, N4",
                    dist = "categorical")

defNeighborhood = defCondition(defNeighborhood, condition = "eye_color=='green'", 
                    formula = "0.2, 0.4, 0.3, 0.1",
                    variance = "N1, N2, N3, N4",
                    dist = "categorical")

defEdu = defCondition(condition = "eye_color=='blue'", 
                    formula = "0.65, 0.25, 0.1",
                    variance = "e1, e2, e3",
                    dist = "categorical")

defEdu = defCondition(defEdu, condition = "eye_color=='red'", 
                    formula = "0.2, 0.3, 0.5",
                    variance = "e1, e2, e3",
                    dist = "categorical")

defEdu = defCondition(defEdu, condition = "eye_color=='green'", 
                    formula = "0.3, 0.4, 0.3",
                    variance = "e1, e2, e3",
                    dist = "categorical")

defIncome = defCondition(condition = "education=='e1'", 
                    formula = 500,
                    variance = 100**2,
                    dist = "normal")

defIncome = defCondition(defIncome, condition = "education=='e2'", 
                    formula = 750,
                    variance = 125**2,
                    dist = "normal")

defIncome = defCondition(defIncome, condition = "education=='e3'", 
                    formula = 1500,
                    variance = 250**2,
                    dist = "normal")

efIncomeBiased = defCondition(condition = "eye_color=='blue'", 
                    formula = "0.9*income_raw",
                    dist = "nonrandom")

defIncomeBiased = defCondition(defIncomeBiased, condition = "eye_color!='blue'", 
                    formula = "income_raw",
                    dist = "nonrandom")
d
defIncomeBiasedGender = defCondition(condition = "gender==0", 
                        formula = "0.95*income_biased",
                        dist = "nonrandom")

defIncomeBiasedGender = defCondition(defIncomeBiasedGender, condition = "gender==1", 
                        formula = "income_biased",
                        dist = "nonrandom")

df3 = addCondition(defNeighborhood, df2, newvar="neighborhood")
df3 = addCondition(defEdu, df3, newvar="education")
df3 = addCondition(defIncome, df3, newvar="income_raw")
df3 = addCondition(defIncomeBiased, df3, newvar="income_biased")
df3 = addCondition(defIncomeBiasedGender, df3, newvar="income_biased2")

In [ ]:
defNeighborhood

In [ ]:
df3.groupby('eye_color')['education'].value_counts()

In [ ]:
df3.groupby('eye_color')['income_raw'].mean()

In [ ]:
m = df3['income_biased2'].mean()
s = df3['income_biased2'].std()
df3['income_standard'] = df3['income_biased2'].apply(lambda x: (x - m) / s)

In [ ]:
df3.groupby('eye_color')['income_standard'].mean()

Income is the only factor directly influencing approval

In [ ]:
defTarget =  defData(varname = "approval", dist = "binary", formula = "0.5+(income_standard/10)",)
defTarget

But, income is biased based on previous conditions

In [ ]:
df3 = addColumns(defTarget, df3)

In [ ]:
df3

In [ ]:
os.getcwd()

In [ ]:
df3.to_csv('/Users/davidcruz/Desktop/Side_Projects/simulacra-fake-data/approval-fake.csv')

In [ ]:
df3.groupby('eye_color')['approval'].value_counts(normalize=True)

In [ ]:
df3.groupby('education')['approval'].value_counts(normalize=True)

In [ ]:
df3['const'] = 1
mod = sm.GLM(df3['approval'], pd.get_dummies(df3.drop(['id', 'approval', 'income_raw', 
            'income_biased', 'income_biased2'], axis = 1)),
            family=sm.families.Binomial())
res = mod.fit()
print(res.summary())

In [ ]:
coefasProb(-0.0563)

In [ ]:
coefasProb(0.0305)

In [ ]:
coefasProb(0.0075)

In [ ]:
df3['const'] = 1
mod = sm.GLM(df3['approval'], pd.get_dummies(df3.drop(['id', 'approval', 'income_raw', 'eye_color',
            'income_biased', 'income_biased2'], axis = 1)),
            family=sm.families.Binomial())
            
res = mod.fit()
print(res.summary())

In [ ]:
df3.columns

In [ ]:
pd.get_dummies(df3.drop(['id', 'approval', 'income_raw', 
            'income_biased', 'income_biased2'], axis = 1))

In [ ]:
from sklearn.utils import compute_sample_weight, resample

from sklearn.model_selection import  train_test_split, StratifiedKFold, GridSearchCV, KFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, Ridge, RidgeCV, RidgeClassifier, RidgeClassifierCV, LinearRegression, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold
# from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn import tree

from sklearn.metrics import classification_report, mean_squared_error, r2_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline

In [ ]:
X = df3.drop(['id', 'approval', 'income_raw',
            'income_biased', 'income_biased2'], axis = 1)

In [ ]:
y = df3['approval']

In [ ]:
# establish categorical variables in X
categorical = X.dtypes == object
# preprocessing pipeline
preprocess = make_column_transformer(
    (StandardScaler(), ~categorical),
    (OneHotEncoder(handle_unknown = 'ignore'), categorical)
)

cv = KFold(n_splits = 5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model = make_pipeline(preprocess, LogisticRegression())
model.fit(X_train, y_train);

In [ ]:
cat_names = ['eye_color', 'neighborhood', 'education']
final_feature_names = categorical[~categorical].index.values.tolist() + \
                      preprocess.transformers_[1][1].get_feature_names_out(cat_names).tolist()

coefs = model.named_steps['logisticregression'].coef_[0]

featureImportance_df = pd.Series(data = coefs, 
                                 index = final_feature_names, 
                                 name = 'coefs').\
                        sort_values(ascending = False, key = np.abs)

In [ ]:
# setting a y_axis variable allows to flip chart for easier reading        
yax = np.arange(len(featureImportance_df))

fig, ax = plt.subplots(1, 1, figsize = (8,8))

ax.plot(featureImportance_df, yax, 'o', c='r')
ax.set_yticks(range(len(featureImportance_df)))
ax.yaxis.tick_right()
ax.set_yticklabels(featureImportance_df.index,rotation = 0)
ax.set_title('lasso'.capitalize()+' Coefficients\nTarget Variable Name: '+str(y.name))
ax.axvline(0)

plt.show()

In [ ]:
df3[df3['eye_color'] == 'blue']['neighborhood'].value_counts(normalize=True)

In [ ]:
X_train

In [ ]:
X_train['gender'].value_counts()[X_train['gender'].value_counts() > 1].reset_index()['gender']

In [ ]:
y_pred = model.predict(X_test)